<a href="https://colab.research.google.com/github/leem928/AI_Project/blob/main/easyOCR%EC%A0%84%EC%B2%98%EB%A6%AC%EC%99%80%EB%9D%BD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import zipfile
import os

# 🔹 경로 설정 (압축파일 경로)
json_zip = "/content/json.zip"            # JSON 압축파일 경로
img_zip = "/content/images_more.zip"      # 이미지 압축파일 경로

# 🔹 압축 해제 폴더 생성
json_dir = "/content/json"
img_dir = "/content/images_more"
os.makedirs(json_dir, exist_ok=True)
os.makedirs(img_dir, exist_ok=True)

# 🔹 JSON 압축 해제
with zipfile.ZipFile(json_zip, 'r') as zip_ref:
    zip_ref.extractall(json_dir)
print(f"✅ JSON 압축 해제 완료: {json_dir}")

# 🔹 이미지 압축 해제
with zipfile.ZipFile(img_zip, 'r') as zip_ref:
    zip_ref.extractall(img_dir)
print(f"✅ 이미지 압축 해제 완료: {img_dir}")

# 🔹 결과 확인
print("\n📁 JSON 폴더 내 파일 샘플:")
print(os.listdir(json_dir)[:5])

print("\n📁 이미지 폴더 내 파일 샘플:")
print(os.listdir(img_dir)[:5])

✅ JSON 압축 해제 완료: /content/json
✅ 이미지 압축 해제 완료: /content/images_more

📁 JSON 폴더 내 파일 샘플:
['json']

📁 이미지 폴더 내 파일 샘플:
['images_more']


In [2]:
!pip install easyocr

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 96.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.7/180.7 kB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 963.8/963.8 kB 71.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 292.1/292.1 kB 30.6 MB/s eta 0:00:00


In [3]:
import os
import re
import json
import pandas as pd
from tqdm import tqdm

# ==============================
# 📁 경로 설정
# ==============================
json_dir = "/content/json/json"  # JSON 폴더 경로
output_csv = "/content/ground_truth.csv"  # 출력 CSV 경로

# ==============================
# 🔢 자연수 정렬 함수
# ==============================
def natural_key(s: str):
    """파일 이름의 숫자 기준으로 정렬 (cosmetics_2.json → cosmetics_10.json 순서 맞춤)"""
    return [int(t) if t.isdigit() else t.lower() for t in re.split(r'(\d+)', s)]

# ==============================
# 🔍 JSON에서 type=2 텍스트 추출 함수
# ==============================
def extract_type2_text(json_path):
    """JSON에서 type=2의 모든 텍스트를 연결해서 반환"""
    with open(json_path, "r", encoding="utf-8") as f:
        data = json.load(f)

    texts = []
    for ann in data.get("annotations", []):
        for poly in ann.get("polygons", []):
            if str(poly.get("type")) == "2" and "text" in poly:
                txt = str(poly["text"]).strip()
                if txt:
                    texts.append(txt)
    return " ".join(texts).strip()

# ==============================
# 📄 전체 JSON 파일 불러오기 (정렬 포함)
# ==============================
json_files = sorted(
    [f for f in os.listdir(json_dir) if f.endswith(".json")],
    key=natural_key  # ✅ 여기서 숫자 순서로 정렬
)

results = []
for file in tqdm(json_files, desc="📑 JSON 파일 처리 중"):
    json_path = os.path.join(json_dir, file)
    gt_text = extract_type2_text(json_path)
    if gt_text:
        results.append({
            "filename": file,
            "GT_text": gt_text
        })

# ==============================
# 💾 CSV 저장
# ==============================
df = pd.DataFrame(results)
df.to_csv(output_csv, index=False, encoding="utf-8-sig")

print(f"✅ 완료! {len(df)}개 파일의 정답(type=2) 텍스트를 추출했습니다.")
print(f"📁 저장 경로: {output_csv}")
display(df.head(15))

📑 JSON 파일 처리 중: 100%|██████████| 100/100 [00:00<00:00, 354.56it/s]


✅ 완료! 100개 파일의 정답(type=2) 텍스트를 추출했습니다.
📁 저장 경로: /content/ground_truth.csv


,filename,GT_text
0,cosmetics_00007.json,"| 전성분 | 정제수, 1,2-헥산다이올, 프로판다이올, 부틸렌글라이콜, 소듐하이알..."
1,cosmetics_00022.json,"전성분 스테아레이트, 씨드, 이알루로네이트, 리놀레닉애씨드, 정제수, 프로판다이올,..."
2,cosmetics_00025.json,"● 전성분: 소듐팔메이트, 벤질벤조에이트, 리모넨, 비에이치티, 소듐팜커넬레이트, ..."
3,cosmetics_00026.json,"[전성분] 정제수, 병풀추출물(580ppm), 변성알코올, 글루코노락톤, 부틸렌글라..."
4,cosmetics_00031.json,"전성분 정제수, 에탄올, 부틸렌글라이콜, 해수, 만 델릭애씨드, 소듐시트레이트, 가..."
5,cosmetics_00039.json,"전성분 정제수, 석시네이트, 펄라이트, 트, 라우릴글루코사이드, 다이소듐코코암포다이..."
6,cosmetics_00041.json,"전성분 정제수, 다이아세테이트, 레이트코폴리머, 구슬나무잎추출물, 라우릴베타인, 소..."
7,cosmetics_00045.json,"【주요성분】 치오글리콜산 전성분 정제수, 세테아릴알코올, 미네랄오일, 우레아, 칼슘..."
8,cosmetics_00047.json,"【전성분】 에틸헥시팔미테이트, 오일, 스위트아몬드오일, 올리브오일, 토코페릴아세테이..."
9,cosmetics_00049.json,"전성분: 록사이드, 호호바씨오일, 칠이오논, 제라리올 정제수, 세테아레스-20, 카..."


In [4]:
import os
import re
import json
import cv2
import numpy as np
import easyocr
from tqdm import tqdm
import matplotlib.pyplot as plt

# ==============================
# 📁 경로 설정
# ==============================
json_dir = "/content/json/json"
image_dir = "/content/images_more/images_more"
output_dir = "/content/ocr_visualized"
os.makedirs(output_dir, exist_ok=True)

# ==============================
# 🚀 EasyOCR 초기화
# ==============================
reader = easyocr.Reader(['ko'], gpu=True)

# ==============================
# 🔢 자연수 정렬 함수 정의
# ==============================
def natural_key(s: str):
    # 예: cosmetics_2.json, cosmetics_10.json 순서 정렬
    return [int(t) if t.isdigit() else t.lower() for t in re.split(r'(\d+)', s)]

# 🔝 상위 N개만 테스트하고 싶으면 지정 (None이면 전체)
top_n = None  # 예: 10 으로 바꾸면 상위 10개만 실행

# ==============================
# 🔍 JSON 파일 목록 정렬
# ==============================
json_files = [f for f in os.listdir(json_dir) if f.endswith(".json")]
json_files = sorted(json_files, key=natural_key)

if top_n is not None:
    json_files = json_files[:top_n]

# ==============================
# ✳️ JSON 기반 type=2 OCR + 시각화
# ==============================
for file in tqdm(json_files):
    json_path = os.path.join(json_dir, file)
    with open(json_path, "r", encoding="utf-8") as f:
        data = json.load(f)

    img_name = data.get("name") or file.replace(".json", ".jpg")
    base_name = os.path.splitext(img_name)[0]

    # 원본 이미지 경로 찾기
    for ext in [".jpg", ".jpeg", ".png"]:
        img_path = os.path.join(image_dir, base_name + ext)
        if os.path.exists(img_path):
            break
    else:
        continue

    img = cv2.imread(img_path)
    if img is None:
        continue

    # 시각화용 복사본
    vis_img = img.copy()
    ocr_texts = []

    # ==============================
    # 🧠 type=2 polygon OCR 수행
    # ==============================
    for ann in data.get("annotations", []):
        for poly in ann.get("polygons", []):
            if str(poly.get("type")) != "2" or "points" not in poly:
                continue

            points = np.array(poly["points"], np.int32)
            x_min, y_min = np.min(points[:, 0]), np.min(points[:, 1])
            x_max, y_max = np.max(points[:, 0]), np.max(points[:, 1])
            crop = img[y_min:y_max, x_min:x_max]

            # OCR 수행
            try:
                results = reader.readtext(crop, detail=0, paragraph=True)
                text = " ".join(results)
                if text.strip():
                    ocr_texts.append(text)
            except Exception as e:
                tqdm.write(f"⚠️ OCR 오류: {file} → {e}")
                continue

            # 바운딩박스 시각화
            cv2.polylines(vis_img, [points], isClosed=True, color=(0, 0, 255), thickness=2)
            cv2.putText(
                vis_img,
                "type2",
                (x_min, max(0, y_min - 10)),
                cv2.FONT_HERSHEY_SIMPLEX,
                0.6,
                (0, 0, 255),
                2,
                cv2.LINE_AA,
            )

    # ==============================
    # 💾 시각화 결과 저장
    # ==============================
    save_path = os.path.join(output_dir, f"{base_name}_vis.png")
    cv2.imwrite(save_path, vis_img)

    # ==============================
    # 📄 OCR 결과 출력 (정렬 순서 유지)
    # ==============================
    tqdm.write(f"📄 {base_name}.png OCR 결과:")
    tqdm.write(" ".join(ocr_texts))
    tqdm.write("=" * 80)

# ==============================
# ✅ 처리 완료
# ==============================
tqdm.write(f"\n✅ 완료! 시각화 이미지는 {output_dir} 폴더에 순서대로 저장됨.")

Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.1% Complete

  1%|          | 1/100 [00:02<03:21,  2.04s/it]

📄 cosmetics_00007.png OCR 결과:
전성분 정제수 1,2-핵산다이올 프로관다이올 부딪렌들라이클 소문하이알루로네이트 복시나무잎추출물 사과수 소임주출물 쌀추출물 다이프로필렌글라이클 병나무잎추출물 힘버드나무 매실주출물 판터늘 트로메타민 아크렇레 껍질추출물 회화나나무뿌리주출물 세라마이드엔피 이올 (12-14피렉스-12 이트010-30알길아크락레이트크로스플리머 예텔핵실긁리세린 다이소둥이디티에이 향료 카보머 메략프로관다


  2%|▏         | 2/100 [00:03<02:39,  1.63s/it]

...

📄 cosmetics_00507.png OCR 결과:
유호성분 피마자유 피이자 8 향료 알루미늄글로로하이드레이트액50이 하드록시에팅설물로오스 기타철가제_ 2 올립도데카올 변성알코올돌리옥시에탤렌경화 시트로산 정제수


 99%|█████████▉| 99/100 [01:35<00:01,  1.18s/it]

📄 cosmetics_00519.png OCR 결과:
전성분 소톱질매이트 소둥잠러엘레이터 정제수 깜국 다이소문라우 소톱홀로라이드; 향료 날애씨드 킬설포석시네이틈 폐트출라툼_ 글리세린 '올리브 부)페닐에털 테꼬소문 핵실신남일 리날록: 오일019 6) 프로피오날 테트라소문이디티에이; 시슬로렉올 에티드로의애씨드 에티드로데이트 하이드록시시튿로빛알 벤질딩조에이트 리모년트 등세205호 '코카미도-로필베타인 '시어버터 청색4이호: 형색_0호- 아르간거널오일 코코봇 소움 '호호바씨오일 해바라기씨오일 아자오일= 벤조에이트느 토고제물 전성문 소톱팔메이튿_ 선원-커엘레이트 '정제속 깜래 다이소문라-우 소톱렉화이드 넷애씨드 킬설포적시네이트 폐트물화툼_ '글리세린 '올 부처페닐메달 시트로날올 '오일0790) 리날물 '핵실신남알 프로피오날 테트라소움이디티에이_ 에티드로님애씨드 에티드로데이트 하이드록시시트로델알 벤질벤조에이트트 시어버터= '코카미도프로필베타인; 청쓰_어호 확색2이호= 아르간러널오일트 코코대 소염 '호호바씨오일 '해바라-씨오일; 야자오일 벤조에이트 '토코페록 전성분 '문질레이끝 소컴커엘레이트 '청제수 깜켜 디이소움문라우 소톱물바라이드 향료 텔애 드: 텍설요_시데이트= 폐투물리늄: '글리세린: 올리브 '부립페닐레들 '핵실신남말 리날출 오왜019 6) 흐중피오날 테트리소문이디리에이  테@소든 지슬로렉올극 얘티드로니애씨드_ 메티트로네이트 [톤 헤미트록: [묻 벤집멘초에이트 리모예= 등색2053' 코카미도;로편에타인 '세어리터 청쓰_어호; 침색히로 아르간거넷오임: 코코넷 소문 호호바]오임 해바라기씨오입= 야자회임 반조에이블 '토조때터 전성분 소톱필헤이트 소문전커엘레이트 다이소문라우 소톱홀로라이드; 향료- 텔애치트 킬설프로시네이트 폐트물리롬 글리서리 울리다 '부탤페닐메트 힘실신남알 리날물 오101976) 프로피오날 테트라소둥이디티에이 테트소둠 시트로닐움 리모번 벤집반초에이트 에티드로니애씨드 에티드로데이트 히이등록시시트로할 십어버터: 코카미로프로필베타인 청쓰스어초 형색 호 아르간거널

100%|██████████| 100/100 [01:36<00:00,  1.04it/s]

📄 cosmetics_00524.png OCR 결과:
전성분 세테아길일코올 이드 아이소프로필미리스테이트 코돌리머 양산딸기주출{25마가) 세테아레스-20 우리이59) 치오글리롬산갈흉액(71.39) [가악) 리키이트 도록사이드 조한황료 (버스티베리스 해바라기씨오일 아크렇레이트코`리머 비치스트로배리주출물(259가럼) 마그네숨알루미늄실 정제수 갈숨하이드록사 글리세질스테아레이트 브이피/에이코신 라즈배리주출국25 포다운하이 16077+ 셋제 티타늄디오사이드 0496(씨아이 77891) 최

✅ 완료! 시각화 이미지는 /content/ocr_visualized 폴더에 순서대로 저장됨.


In [5]:
import pandas as pd

# ==============================
# 💾 OCR 결과 CSV 저장 (파일별로 즉시 기록)
# ==============================
ocr_results = []

for file in tqdm(json_files):
    json_path = os.path.join(json_dir, file)
    with open(json_path, "r", encoding="utf-8") as f:
        data = json.load(f)

    img_name = data.get("name") or file.replace(".json", ".jpg")
    base_name = os.path.splitext(img_name)[0]

    # 원본 이미지 경로 탐색
    for ext in [".jpg", ".jpeg", ".png"]:
        img_path = os.path.join(image_dir, base_name + ext)
        if os.path.exists(img_path):
            break
    else:
        continue

    img = cv2.imread(img_path)
    if img is None:
        continue

    ocr_texts = []  # ✅ 여기서 새로 초기화 (파일마다 독립적으로)
    vis_img = img.copy()

    # type=2 OCR 수행
    for ann in data.get("annotations", []):
        for poly in ann.get("polygons", []):
            if str(poly.get("type")) != "2" or "points" not in poly:
                continue

            points = np.array(poly["points"], np.int32)
            x_min, y_min = np.min(points[:, 0]), np.min(points[:, 1])
            x_max, y_max = np.max(points[:, 0]), np.max(points[:, 1])
            crop = img[y_min:y_max, x_min:x_max]

            try:
                results = reader.readtext(crop, detail=0, paragraph=True)
                text = " ".join(results)
                if text.strip():
                    ocr_texts.append(text)
            except Exception as e:
                print(f"⚠️ OCR 오류: {file} → {e}")
                continue

            cv2.polylines(vis_img, [points], isClosed=True, color=(0, 0, 255), thickness=2)

    # 💾 OCR 결과 저장 (각 파일마다 즉시 기록)
    combined_text = " ".join(ocr_texts).strip()
    ocr_results.append({
        "filename": f"{base_name}_vis.png",
        "ocr_text": combined_text
    })

# ✅ 모든 결과를 CSV로 저장
df = pd.DataFrame(ocr_results)
df.to_csv("/content/ocr_texts.csv", index=False, encoding="utf-8-sig")

print("✅ OCR 결과 CSV 저장 완료: /content/ocr_texts.csv")
display(df.head(15))

100%|██████████| 100/100 [01:28<00:00,  1.13it/s]

✅ OCR 결과 CSV 저장 완료: /content/ocr_texts.csv


,filename,ocr_text
0,cosmetics_00007_vis.png,"전성분 정제수 1,2-핵산다이올 프로관다이올 부딪렌들라이클 소문하이알루로네이트 복시..."
1,cosmetics_00022_vis.png,전성뿐 스테아레이트_ 씨드 이알루로데이트 리돌레나애씨드 정제수 프로관다이올 피로스데...
2,cosmetics_00025_vis.png,전성분 소흉메이트 벤질트조에이트 리모넷 비에이치터 소문팔커엘레이트 정제수 유차나무싸...
3,cosmetics_00026_vis.png,[선성 청제운 병운출출물숙8이의[) 변성말되올 불문질도락롬: 분플렌스렉이블 매테라소...
4,cosmetics_00031_vis.png,딱심문 청체문 에컨올 부립레플라이골 해술 훨릭애지운 소둘시트레이트 카체염매주출움 배...
5,cosmetics_00039_vis.png,전성분 정제수 석시네이트 필라이트 트 라우킬금루코사이드 다이소둡고코앉포다이아세테이트...
6,cosmetics_00041_vis.png,며신- 비심 더_어하세'이터 귀| :::출다다; 음}출국목 라컴플베키없 조육바라로활...
7,cosmetics_00045_vis.png,[주요성분) 치오글리록산 전성뷰 정제수 세테야람말코올 미네활오일 우아 갈습하이드록사...
8,cosmetics_00047_vis.png,[전성뷰) 에달렉실팔미테이트 오일 스위드이모드오일 올리브오일 토`페랗아서테이트 미키...
9,cosmetics_00049_vis.png,전성분 록사이드 호호바씨오일 칠이오든: 제라리올 정제수 세테아래스-20 카카오씨드버...


In [6]:
import pandas as pd
import editdistance
from tqdm import tqdm
import re

# ==============================
# 📁 파일 경로
# ==============================
gt_csv = "/content/ground_truth.csv"
ocr_csv = "/content/ocr_texts.csv"
output_csv = "/content/ocr_eval_results.csv"

# ==============================
# 🧩 파일 불러오기
# ==============================
gt_df = pd.read_csv(gt_csv)
ocr_df = pd.read_csv(ocr_csv)

# 파일명 정리 (base 기준으로 merge)
gt_df['base'] = gt_df['filename'].str.replace('.json', '', regex=False)
ocr_df['base'] = ocr_df['filename'].str.replace('_vis.png', '', regex=False)

merged = pd.merge(gt_df, ocr_df, on='base', how='inner')
print(f"🔗 매칭된 파일 수: {len(merged)}개\n")

# ==============================
# 📏 평가 함수 정의
# ==============================
def cer(ref, hyp):
    ref, hyp = ref.replace(" ", ""), hyp.replace(" ", "")
    return editdistance.eval(ref, hyp) / max(1, len(ref))

def wer(ref, hyp):
    ref_words, hyp_words = ref.split(), hyp.split()
    return editdistance.eval(ref_words, hyp_words) / max(1, len(ref_words))

def precision_recall(gt, pred):
    # 단어 단위로 비교 (대소문자 무시)
    gt_tokens = set(re.findall(r"[가-힣A-Za-z0-9]+", gt.lower()))
    pred_tokens = set(re.findall(r"[가-힣A-Za-z0-9]+", pred.lower()))

    if not gt_tokens:
        return 0, 0, 0

    tp = len(gt_tokens & pred_tokens)
    fp = len(pred_tokens - gt_tokens)
    fn = len(gt_tokens - pred_tokens)

    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    f1 = (2 * precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

    return precision, recall, f1

# ==============================
# 📊 파일별 성능 계산
# ==============================
results = []
for _, row in tqdm(merged.iterrows(), total=len(merged)):
    gt_text = str(row['GT_text']).strip()
    ocr_text = str(row['ocr_text']).strip()

    c = cer(gt_text, ocr_text)
    w = wer(gt_text, ocr_text)
    p, r, f1 = precision_recall(gt_text, ocr_text)

    results.append({
        "filename": row['filename_x'],
        "CER": c,
        "WER": w,
        "Precision": p,
        "Recall": r,
        "F1-Score": f1
    })

df_eval = pd.DataFrame(results)
df_eval.to_csv(output_csv, index=False, encoding="utf-8-sig")

# ==============================
# 📈 평균값 출력
# ==============================
print("\n📊 성능 요약 (평균)")
print(f"✅ 평균 CER: {df_eval['CER'].mean():.4f}")
print(f"✅ 평균 WER: {df_eval['WER'].mean():.4f}")
print(f"✅ 평균 Precision: {df_eval['Precision'].mean():.4f}")
print(f"✅ 평균 Recall: {df_eval['Recall'].mean():.4f}")
print(f"✅ 평균 F1-Score: {df_eval['F1-Score'].mean():.4f}")

display(df_eval.head(10))

🔗 매칭된 파일 수: 100개



100%|██████████| 100/100 [00:00<00:00, 2724.55it/s]


📊 성능 요약 (평균)
✅ 평균 CER: 0.4175
✅ 평균 WER: 0.9761
✅ 평균 Precision: 0.2338
✅ 평균 Recall: 0.2481
✅ 평균 F1-Score: 0.2374


,filename,CER,WER,Precision,Recall,F1-Score
0,cosmetics_00007.json,0.283654,0.933333,0.406250,0.382353,0.393939
1,cosmetics_00022.json,0.310056,0.943396,0.259259,0.250000,0.254545
2,cosmetics_00025.json,0.364641,1.000000,0.185185,0.178571,0.181818
3,cosmetics_00026.json,0.703448,1.000000,0.000000,0.000000,0.000000
4,cosmetics_00031.json,0.594937,1.000000,0.102041,0.102041,0.102041
5,cosmetics_00039.json,0.291188,0.972973,0.256410,0.243902,0.250000
6,cosmetics_00041.json,0.896679,1.000000,0.000000,0.000000,0.000000
7,cosmetics_00045.json,0.369369,1.000000,0.171429,0.171429,0.171429
8,cosmetics_00047.json,0.373333,0.909091,0.333333,0.363636,0.347826
9,cosmetics_00049.json,0.222222,0.894737,0.500000,0.500000,0.500000
